# 📓 Lab Notebook 1: Definition of the Workspace
**Author:** Martín Campillo \
**Project:** Algorithmic Trading Bot (ProjectAT) \
**Date:** 21 / 11 / 25

## 🎯 Objective: 
This notebook's objective is to define three of the main pillars of the trading environment:
1. **Market Selection.** 
2. **Asset Selection.**
3. **Exchange Selection.**

# 📈 Market Selection 

The market in which the analisis is going to be made is crucial since every market has his own features. For this project I have chosen the crypto and forex markets. The main reason of this is that both markets offer large amounts of free data, which is best for the objective of this project. 
Now, let's compare both markets features.

## ₿ Crypto vs Forex 💲

| **Feature** | **Forex** | **Crypto** |
|---------|--------|--------|
| **Data avaiability** | High (Broker API) | **High (ccxt)** |
| **Data price** | **Free** | **Free** |
| **Volativity** | **Low** | Very high |
| **Liquidity** | **Very high** | **Very high (top coins)** |
| **Continuity** | 24/5 | **24/7** |
| **Transaction price** | **Very low (variable)** | Low/Medium (fixed)|
| **Volume Data** | No (Decentralized) | **Yes (Centralized)** |

## €/💲 Asset Selection 
The coins that are going to be analized are bitcoin for the crypto market and eur/usd for the forex market. Those coins are the ones that suits best for this proyect because are the top ones in their respective markets.